In [1]:
# Imports
import pandas as pd
import numpy as np
import os

# Preprocessing

In [2]:
# Load data

team_id = '20' #put your team id here
split = 'test_1' # replace by 'test_2' for FINAL submission

df = pd.read_csv('dataset/tweets_train.csv')
df_test = pd.read_csv(f'dataset/tweets_{split}.csv')

In [3]:
df['words_str'] = df['words'].apply(lambda words: ' '.join(eval(words)))
df_test['words_str'] = df_test['words'].apply(lambda words: ' '.join(eval(words)))

# Regression

In [4]:
from transformers import BertTokenizer, BertModel, BertPreTrainedModel, TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaPreTrainedModel, RobertaModel, AutoTokenizer, AutoModel, PreTrainedModel
from transformers import TrainerCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch import optim

2023-08-11 11:58:56.722395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 11:58:57.614093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['words_str'], df['score_compound'], test_size=0.2, shuffle=True, random_state=None)

# Tokenize the input
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer_twitter = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base')

In [7]:
tokenizer = tokenizer_twitter
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
# Create a custom dataset
class RegressionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values.astype('float32') # Convert Series to NumPy array and then to float32

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]) # This is already float32
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = RegressionDataset(train_encodings, train_labels)
val_dataset = RegressionDataset(val_encodings, val_labels)

class BertRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        logits = logits.squeeze()
        loss = None
        if labels is not None:
            loss = torch.sqrt(nn.MSELoss()(logits, labels))
        return (loss, logits) if loss is not None else logits

class RobertaRegression(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        logits = logits.squeeze()
        loss = None
        if labels is not None:
            loss = torch.sqrt(nn.MSELoss()(logits, labels))
        return (loss, logits) if loss is not None else logits
    
class RobertaRegressionTwitter(nn.Module):
    def __init__(self):
        super(RobertaRegressionTwitter, self).__init__()
        self.roberta = AutoModel.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(self.roberta.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        logits = logits.squeeze()
        loss = None
        if labels is not None:
            loss = torch.sqrt(nn.MSELoss()(logits, labels))
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Define a function to compute RMSE
def compute_rmse(eval_pred):
    predictions, labels = eval_pred
    return {'rmse': np.sqrt(mean_squared_error(labels, predictions))}


class RobertaRegressionTwitter_2(nn.Module):
    def __init__(self):
        super(RobertaRegressionTwitter_2, self).__init__()
        self.roberta = AutoModel.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
        self.dropout = nn.Dropout(0.1)
        hidden_size = self.roberta.config.hidden_size

        # Adding an additional hidden layer
        self.hidden_layer = nn.Linear(hidden_size, hidden_size//2)
        
        # Adding L2 regularization (weight decay) to the hidden layer
        self.regularization = nn.LayerNorm(hidden_size//2)
        
        # Final regression layer
        self.regressor = nn.Linear(hidden_size//2, 1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        
        # Passing through the hidden layer with ReLU activation
        hidden_output = self.hidden_layer(pooled_output)
        hidden_output = F.relu(hidden_output)
        
        # Applying Layer Normalization (regularization)
        hidden_output = self.regularization(hidden_output)
        
        logits = self.regressor(hidden_output)
        logits = logits.squeeze()
        
        loss = None
        if labels is not None:
            loss = torch.sqrt(nn.MSELoss()(logits, labels))
        
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

    
    
    
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=4):
        self.patience = patience
        self.best_score = None
        self.early_stop_counter = 0

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        rmse = metrics['eval_rmse']  # Make sure this key matches what's returned by your compute_metrics function
        if self.best_score is None or rmse < self.best_score:
            self.best_score = rmse
            self.early_stop_counter = 0
        else:
            self.early_stop_counter += 1
            if self.early_stop_counter >= self.patience:
                control.should_training_stop = True
        return control
    
class ThresholdEarlyStoppingCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        rmse = metrics['eval_rmse'] # Make sure this key matches what's returned by your compute_metrics function
        if rmse < 0.2:
            control.should_training_stop = True
        return control


In [9]:
model_bert = BertRegression.from_pretrained('bert-base-uncased')
model_roberta = RobertaRegression.from_pretrained('roberta-base')
model_twitter = RobertaRegressionTwitter_2()

Some weights of BertRegression were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.weight', 'regressor.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaRegression were not initialized from the model checkpoint at roberta-base and are newly initialized: ['regressor.weight', 'regressor.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = model_twitter.to(device)

# Define training arguments and trainer
training_args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    learning_rate=0.0001,
    num_train_epochs=100,
    logging_dir='./logs',
    evaluation_strategy='steps', # Evaluate every 'logging_steps'
    logging_steps=100, # Set to evaluate every 100 steps
    weight_decay=0.0001,
)




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_rmse,
    callbacks=[ThresholdEarlyStoppingCallback()],
)


# Train the model
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

/root/miniconda3/envs/mlStuff/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
# Save the model
trainer.save_model('pretrained_models/roberta-base-twitter-regression')

# model = BertRegression.from_pretrained("./path/to/save/directory")

# Test

In [ ]:
# Define a dataset without labels for testing
class RegressionTestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
# Tokenize the test sentences
sentences = list(df_test.words_str.values)
test_encodings = tokenizer(sentences, truncation=True, padding=True)

# Convert to a PyTorch Dataset
test_dataset = RegressionTestDataset(test_encodings)

# Get predictions with the neural network
predictions = trainer.predict(test_dataset)
y_hat_tensor = torch.tensor(predictions.predictions, dtype=torch.float32)

# Convert the predictions back to a numpy array
y_hat = y_hat_tensor.cpu().numpy()

# Save the results with the specified format
directory = 'results'
np.save(os.path.join(directory, f'{team_id}__{split}__reg_pred.npy'), np.squeeze(y_hat))

In [ ]:
# Load 20__test_1__reg_pred.npy

d = np.load('results/20__test_1__reg_pred.npy', allow_pickle=True)
d.shape

(1000,)